In [1]:
import pymongo
from pymongo import MongoClient

In [38]:
client = MongoClient('mongodb://localhost:27017/')
db = client.db_agg
restaurants = db.restaurants
transactions = db.transactions

print('connexion success')

connexion success


In [ ]:
########### Exercice  Aggréger des restaurants

# Q : Créer 1DB restaurant, ajouter 1 collection "restaurants", la remplir avec le fichier restaurants.json.
# Rq: Préciser, lors de l'injection du fichier json, dans la commande "mongoimport" la possibilité de supprimer la collection si elle existe (pour qu'elle soit recréer de nouveau et y injecter de la data fraîche).
# Indice : 25357 restaurant en tout

# mongoimport -d db_agg -c restaurants --drop --file "/Users/eleacarton/Documents/NoSQL/restaurants.json"

In [49]:
# Q : La répartition des restaurants par quartier ?
# Indice : quartier => key 'borough'

resto_borough = { "$group":{"_id": "$borough","nb_restaurant":{"$sum" : 1}}}
tri = {"$sort" : {"_id":1}}

for restaurant in restaurants.aggregate([resto_borough, tri]):
    print(restaurant)

{u'_id': u'Bronx', u'nb_restaurant': 2338}
{u'_id': u'Brooklyn', u'nb_restaurant': 6085}
{u'_id': u'Manhattan', u'nb_restaurant': 10258}
{u'_id': u'Missing', u'nb_restaurant': 51}
{u'_id': u'Queens', u'nb_restaurant': 5656}
{u'_id': u'Staten Island', u'nb_restaurant': 969}


In [54]:
# Q : La répartition des restaurants dont la dernière inspection a donné un grade "C" par quartier ?
# Indice : la dernière inspection : la plus récente, donc la première de la liste !
# key : "grade"

match_grade_C = { "$match" : {"grades.0.grade":"C"}}
proj = {"$project" : {"name":1, "borough":1,"_id":0}}
tri = {"$sort" : {"_id":1}}

for grad in restaurants.aggregate([match_grade_C,proj,repart_resto_borough, tri]):
    print(grad)

{u'_id': u'Bronx', u'nb_restaurant': 27}
{u'_id': u'Brooklyn', u'nb_restaurant': 56}
{u'_id': u'Manhattan', u'nb_restaurant': 83}
{u'_id': u'Queens', u'nb_restaurant': 47}
{u'_id': u'Staten Island', u'nb_restaurant': 7}


In [55]:
# Q : Calculer le score moyen des resto par quartier et trier par score décroissant ?
# Indice : utiliser l'opérateur unwid

eclater_liste = {"$unwind" : "$grades"}
mean_quartier = {"$group" : {"_id":"$borough","moyenne":{"$avg":"$grades.score"}}}
tri = {"$sort": {"moyenne":-1}}

for grad in restaurants.aggregate([eclater_liste, mean_quartier, tri]):
    print(grad)

{u'_id': u'Queens', u'moyenne': 11.634865110930088}
{u'_id': u'Brooklyn', u'moyenne': 11.447723132969035}
{u'_id': u'Manhattan', u'moyenne': 11.41823125728344}
{u'_id': u'Staten Island', u'moyenne': 11.370957711442786}
{u'_id': u'Bronx', u'moyenne': 11.036186099942562}
{u'_id': u'Missing', u'moyenne': 9.632911392405063}


In [ ]:
########### Exercice  Aggréger des transactions 

# Q : Créer une collection "transactions" à partir du fichier transactions.json et répondre aux questions suivantes :

# mongoimport -d db_agg -c transactions --drop --file "/Users/eleacarton/Documents/NoSQL/transactions.json"  --jsonArray

In [56]:
# Q-1. Calculer le montant total des paiements ?
# Indice : key : Payment.Total

transactions = db.transactions
succ = { "$match" : {"Payment.Success":True}}
som = {"$group" : {"_id":"total", "Total":{"$sum":"$Payment.Total"}}}
for pay in transactions.aggregate([succ,som]):
    print(pay)

{u'_id': u'total', u'Total': 1200}


In [76]:
# Q-2. Calculer le montant total par paiment ?

succ = { "$match" : {"Payment.Success":True}}
som = {"$project" : {"_id":1, "Id":1, "Payment.Total": 1}}
for pay in transactions.aggregate([succ,som]):
    print(pay)

{u'_id': ObjectId('60101b973446d1767d11cb0b'), u'Id': 102, u'Payment': {u'Total': 400}}
{u'_id': ObjectId('60101b973446d1767d11cb0c'), u'Id': 101, u'Payment': {u'Total': 400}}
{u'_id': ObjectId('60101b973446d1767d11cb0d'), u'Id': 100, u'Payment': {u'Total': 400}}


In [58]:
#Q-3. Calculate total payments (Payment.Total) for each payment type (Payment.Type) ?

succ = { "$match" : {"Payment.Success":True}}
som = {"$group" : {"_id":"$Payment.Type", "Total":{"$sum":"$Payment.Total"}}}
for pay in transactions.aggregate([succ,som]):
    print(pay)

{u'_id': u'Credit-Card', u'Total': 800}
{u'_id': u'Debit-Card', u'Total': 400}


In [61]:
# Q-4. Trouver l'Id le plus élevé.

sort = {"$sort":{"Id": -1}}
limit = {"$limit": 1}
for idele in transactions.aggregate([sort,limit]):
    print(idele)

{u'Note': u'Payment is Null', u'Transaction': [{u'ItemId': u'a100', u'price': 200}, {u'ItemId': u'a110', u'price': 210}], u'Name': u'Dylan', u'Payment': None, u'Subscriber': True, u'TransactionId': u'tran4', u'_id': ObjectId('60101b973446d1767d11cb0a'), u'Id': 103}


In [81]:
# Q-5. Find the max price (Transaction.price) ?

transactions = db.transactions
succ = { "$match" : {"Payment.Success":True}}
som = {"$group" : {"_id":1, "Max price":{"$max":"$Transaction.price"}}}
for pay in transactions.aggregate([succ,som]):
    print(pay)

{u'Max price': [200, 210], u'_id': 1}


In [ ]:
########### distinct	
# Q : Age unique => distinct ?

# Dans le shell mongo:
# db.people.distinct("age")
# S : [ 20, 35, 60 ]

# Q : Peut-on faire un distinct sur plusieurs key ?

# Dans le shell mongo:
# db.people.distinct("name", "age")
# R : Ce n'est pas possible de faire un distinct sur plusieurs key => Il faut passer par map reduce !